In [2]:
import Pkg; Pkg.activate(joinpath(@__DIR__, "..")); Pkg.instantiate();
using RobotZoo: YakPlane
import RobotDynamics
using RobotDynamics: state_dim, control_dim
using ForwardDiff
using StaticArrays
using LinearAlgebra
using Rotations
using Printf
using Test
using TrajOptPlots
using JLD2
const RD = RobotDynamics
include("utils.jl")
include("airplane.jl")
const isautograder = @isdefined autograder;
const resfile = joinpath(@__DIR__,"q1.jld2")

  Activating environment at `c:\Users\ADMIN\Documents\GitHub\hw4\Project.toml`
Precompiling project...
  ✓ OpenSSL_jll
  ✓ FFMPEG_jll
  ✓ FFMPEG
  ✓ SpecialFunctions
  ✓ DiffRules
  ✓ GeometryBasics
  ✓ MeshIO
  ✓ ForwardDiff
  ✓ SparseDiffTools
  ✓ MeshCat
  ✓ RobotDynamics
  ✓ RobotZoo
  ✓ TrajectoryOptimization
  ✓ Altro
  ✓ TrajOptPlots
  15 dependencies successfully precompiled in 61 seconds (143 already precompiled)


"c:\\Users\\ADMIN\\Documents\\GitHub\\hw4\\src\\q1.jld2"

# Q1: iLQR with Quaternions (35 pts)
In this problem you'll adapt the iLQR algorithm you wrote in the last homework to work with 3D rotations, using the methods we talked about in class. We'll also see one of the drawbacks of DDP-based methods like iLQR: the need for good initialization. 

In this problem we'll be designing an aerobatic maneuver for a small-scale model airplane. The dynamics you'll be using use data extracted from real-world wind-tunnel testing. Since the dynamics are fairly complex, we'll skip describing them in detail. Instead we'll simply describe the state and control inputs:

$$ x = \begin{bmatrix}
    p \\ q \\ v \\ \omega
\end{bmatrix}, \;\; 
u = \begin{bmatrix}  
    u_\text{thr} \\ u_\text{ail} \\ u_\text{elev} \\ u_\text{rud}
\end{bmatrix}
$$
where $p \in \mathbb{R}^3$ is the position of the center of mass in the world frame, $q \in \mathbb{S}^3$ is the unit quaternion mapping vectors in the body frame to the world frame, $v \in \mathbb{R}^3$ is the linear velocity in the world frame, and $\omega \in \mathbb{R}^3$ is the angular velocity in the body frame. The controls are all scalar values ranging from 0-255 for the thrust, aileron, elevator, and rudder control surfaces.

If you want to look at the system dynamics, check out [this file](https://github.com/RoboticExplorationLab/RobotZoo.jl/blob/master/src/yak_plane.jl).


In [3]:
# Dynamics API
let
    model = YakPlane(UnitQuaternion)
    x,_ = rand(model)  # generate a random state and control input (with the orientation sampled correctly)
    u = Vector{Float64}(rand(0:255, 4))
    t  = 0.0  # time
    dt = 0.1  # time step
    
    # continuous dynamics
    dynamics(model, x, u)
    
    # discrete dynamics
    xnext = discrete_dynamics(model, x, u, t, dt);
    
    A,B = discrete_jacobian(model::YakPlane, x, u, t, dt)
    
end;  # wrap in let/end to avoid polluting the global state

## Part (a): Finding the trim conditions (7 pts)
To get a good answer out of shooting methods like iLQR that rely on a dynamically feasible initial guess, it's often a good idea to find some sort of "stable" initial trajectory. For this problem, we'll simply find the "trim" conditions for level flight, or the controls need to maintain level flight with zero acceleration. In principle, this is going to be very similar to the problem in the HW1 where we found the equilibrium state for the quadruped, except that here we're only constraining the accelerations (not velocities) to be zero.

To find the trim conditions, we'll solve the following optimization problem using Gauss-Newton:
$$
\begin{align}
& \text{minimize}_{u} && \frac{1}{2} (u - u_\text{guess})^T (u - u_\text{guess}) \\
& \text{subject to} && f_\text{accel}(x_\text{trim},u) = 0
\end{align}
$$
where $u_\text{guess}$ in an initial guess for the controls, $x_\text{trim}$ is the state vector (both position and velocity) we want to design our trim controls for. In our case, this will be level flight with a forward velocity along the x-axis of 5 m/s. $f_\text{accel}$ is a function that returns the linear and angular accelerations. You can just pull these out of a call to the continuous-time dynamics function.

In [70]:

function kkt_conditions(model::YakPlane, x, u, lambda , u_guess)
    # Get initial state from the model (if you need it)
    ∇ₓL = zero(x)
    ∇ᵤL = zero(u)
    c = zero(x)

    c = dynamics(model, x, u)[end-5:end,:]

    A = ForwardDiff.jacobian(dx-> dynamics(model,dx,u),x)
    B = ForwardDiff.jacobian(du-> dynamics(model,x,du),u)

    A_trim = A[end-5:end,:]
    B_trim = B[end-5:end,:]

    ∇ᵤL = B_trim'*lambda + u - u_guess


    return [∇ᵤL ; c] # 2x1
end

kkt_conditions (generic function with 2 methods)

In [77]:
function kkt_jacobian(model::YakPlane, x, u, lambda, A, B ;ρ=1e-5)
    
    l,m = size(B)
    #n = 6 # States
    #m = 4 # Controls
    #l = 6 # Constraints

    
    H = zeros(m+l,m+l)

    H[1:m,1:m] .= I(m) .+ ρ
    H[1:m,m+1:end] .= B'
    H[m+1:end,1:m] .= B
    H[m+1:end,m+1:end] .= H[m+1:end,m+1:end] - I(l)*ρ

    return Symmetric(H,:L) #2x2
end

kkt_jacobian (generic function with 2 methods)

In [80]:
# TASK: implement get_trim to find the trim controls using Newton's method (7 pts)
"""
    get_trim(model::YakPlane, x_trim, u_guess)

Calculate the trim controls for the YakPlane model, given the position and velocity specified by `x_trim`.
Find a vector of 4 controls close to those in `u_guess` that minimize the accelerations on the plane.
"""
function get_trim(model::YakPlane, x_trim, u_guess;
        verbose = false,
        tol = 1e-4,
        iters = 100,
    )

    u_trim = copy(u_guess)
    lambda = zeros(6)

    xdot = dynamics(model,x_trim,u_trim)

    for i in 1:iters
        A = ForwardDiff.jacobian(dx-> dynamics(model,dx,u_trim),x_trim)
        B = ForwardDiff.jacobian(du-> dynamics(model,x_trim,du),u_trim)

        A_trim = A[8:13,:] # 6,13
        B_trim = B[8:13,:] # 4,1

        res = kkt_conditions(model,x_trim,u_trim,lambda,u_guess)
        #@show(size(res))

        hess = kkt_jacobian(model,x_trim, u_trim, lambda, A_trim, B_trim ; ρ=1e-5)
        #@show(size(hess))

        delz = hess\-res
        #@show(size(delz))

        #@show(size(u_trim))
        u_trim .= u_trim + delz[1:4]

        #@show(size(lambda))
        lambda .= lambda + delz[5:end]
        
        #@show(i)

        if norm(res) < tol
            @show(i)
            break
        end
        

    end
    
    # TODO: Use Newton's method to find the trim controls
    #  TIP: Look at your solution the quadruped problem on HW1
    #  TIP: You should have 4 decision variables and 6 constraints
    #  TIP: You don't need to do anything special due to the rotation, just do normal (Gauss) Newton
    return u_trim
end


get_trim

In [82]:
# Test that the method finds zero acceleration
@testset "Part a" begin                                                           # POINTS = 7
    model = YakPlane(UnitQuaternion)
    x0 = SA[
        0,0,0, 
        -6.114599630220157e-8, 0.997155939027903, 0.0, 0.07536599539167085,
        5,0,0,
        0,0,0
    ]
    u_guess = fill(124.0,4)
    utrim = get_trim(model, x0, u_guess)
    
    # Make sure the accelerations are small
    #@show(x0)
    #@show(dynamics(model, x0, utrim)[8:13])
    @test norm(dynamics(model, x0, utrim)[8:13]) < 1e-3                           # POINTS = 5
    @test utrim ≈ load(resfile, "utrim") rtol=1e-3                                # POINTS = 2
end; # wrap in begin/end to avoid polluting the global state with variables

i = 13
Test Summary: | Pass  Total
Part a        |    2      2


## Part (b): Computing the Error State (5 pts)
As we learned in class, we will compute the backward pass of iLQR on the error state, which for a single rigid body like the airplane will be an element of $\mathbb{R}^{12}$. We need a method for converting between our state vector in $\mathbb{R}^{9} \times \mathbb{S}^3$ into an error state. 

Before computing the error state, we need a few methods to work with quaternions. We'll need a way to multiply two quaternions. The easiest way to do this is by defining the linear operator $L(q_1) q_2$. Define the `lmult` method below.

In [155]:
# TASK: implement lmult (1 pt)

"""
    lmult(q)

Return the 4x4 matrix `L` such that `L(q1)*q2` is equivalent to the quaternion multiplication `q1*q2`,
where `q1` and `q2` follow the Hamilton convention (i.e. q[1] is the scalar part and q[2:4] is the vector part).
"""
function lmult(q)
    # TODO: calculate L(q)
    S = zeros(4,4)
    
    s = q[1]
    vector = q[2:4]

    w1 = q[2]
    w2 = q[3]
    w3 = q[4]

    vhat = [s -w3 w2; w3 s -w1 ; -w2 w1 s]

    S[1,1] = s
    S[1,2:4] = -vector'
    S[2:4,1] = vector
    S[2:4,2:4] = vhat
    #@show(S)

    return S
end

lmult

There are many ways to represent a three-parameter error quaternion, but we'll use the *Cayley Map*, which we've found to work best and is very computationally efficient. This is equivalent to converting the quaternion difference to a Rodrigues Parameter:
$$ g = \frac{1}{q_s} \begin{bmatrix} q_x \\ q_y \\ q_z \end{bmatrix} $$

Using the Cayley map, compute the error state, given two states $x$ and $x_0$. This is equivalent to $x - x_0$ for vector (Euclidean) states. Remember that the difference between two quaternions is just $q_0^\dagger q$ which is the same as $L(q_0^\dagger) q = L(q_0)^T q$.

In [175]:
# TODO: Implement the state_error function (2 pts)
"""
    state_parts(model::YakPlane)
Give a tuple of indices, `ip,iq,iv,iw` for extracting pieces of the state vector:
* `x[ip]` returns the 3 positions in the world frame
* `x[iq]` returns the 4 elements of the unit quaternion
* `x[iv]` returns the 3 linear velocities in the world frame
* `x[iw]` returns the 3 angular velocities in the body frame
"""
function state_parts(model::YakPlane)
    ip = SA[1,2,3]
    iq = SA[4,5,6,7]
    iv = SA[8,9,10]
    iw = SA[11,12,13]
    return ip, iq, iv, iw
end


function state_error(model::YakPlane, x, x0)
    # TODO: Compute the error state between x and x0
    dx = zeros(length(x)-1)
    ip,iq,iv,iw = state_parts(model)

    dx[ip] = x[ip] - x0[ip]

    diff = lmult(x0[iq])'*x[iq]
    qs,qx,qy,qz = diff[1],diff[2],diff[3],diff[4]
    dx[4],dx[5],dx[6] = qx/qs, qy/qs, qz/qs

    dx[7:9] = x[iv] - x0[iv]
    dx[10:12] = x[iw] -x0[iw]
    
    return dx
end

state_error (generic function with 1 method)

Now that we have a function to compute the error state, we're also going to need it's Jacobian (no surprise there). Remember that the *attitude Jacobian* is the Jacobian of the quaternion difference, and is given as:
$$
G(q) = \begin{bmatrix}
    -q_v^T \\ q_s I_3 + \hat{q}_v
\end{bmatrix}
$$
where $q_s \in \mathbb{R}$ and $q_v \in \mathbb{R}^{3}$ are the scalar and vector part of the quaternion, respectively, and $\hat{x}$ is the skew-symmetric matrix operator:
$$
\hat{x} = \begin{bmatrix}
 0   & -x_3 & x_2 \\
 x_3 &  0   &-x_1 \\
-x_2 &  x_1 & 0   \\
\end{bmatrix}
$$

In [186]:
# TODO: Implement the following methods
#       1. attitude_jacobian     (1 pt)
#       2. state_error_jacobian  (1 pt)

"""
    attitude_jacobian(q)

Return the 4x3 attitude Jacobian for the unit quaternion q following the Hamilton convention.
The attitude Jacobian is the Jacobian of `q*ϕ(dq)` with respect to `dq`,
where `ϕ` transforms the infinitessimal rotation `dq` to a unit quaternion.
"""
function attitude_jacobian(q)

    s = q[1]
    vector = q[2:4]

    w1 = q[2]
    w2 = q[3]
    w3 = q[4]

    vhat = [s -w3 w2; w3 s -w1 ; -w2 w1 s]

    L = [-vector';vhat]
    #return @SMatrix zeros(4,3)
    return L
end

"""
    state_error_jacobian(model, x)

Return the Jacobian of the `state_error` function, with respect to the state `x`.
"""
function state_error_jacobian(model, x)

    S = zeros(length(x), length(x)-1)
    ip,iq,iv,iw = state_parts(model)
    S[ip,ip] = I(3)
    S[iq,4:6] = attitude_jacobian(x[iq])
    S[iv,7:9] = I(3)
    S[iw,10:12] = I(3)
    return S
end

state_error_jacobian

In [187]:
# Test the attitude and state error Jacobians
@testset "Part b" begin                                           # POINTS = 5
    model = YakPlane(UnitQuaternion)
    q1 = [0.15923376998393293, 0.26188238416859005, 0.944750125601984, 0.11623004574512731]
    q2 = [-0.511916585919045,  0.03735280673079172, -0.036349007931561726, -0.8574525797457913]
    x1 = [0.989607214000322, 0.51362326582473, 0.3616831284706765, -0.6336197238671922, 0.01573491125215133, -0.7172154648116752, -0.28962119247272433, 0.008365202831213558, 0.5515476660019731, 0.19996988824486706, 0.25027308097388, 0.8555278096133965, 0.9134644393324214]
    x2 = [0.45493888030931307, 0.8214867872560794, 0.587845774939719, 0.0859756782429905, -0.357328766626224, -0.4039767922209037, -0.8376915223630642, 0.274895624769145, 0.40424715886322726, 0.7686052831243393, 0.1958635140022562, 0.33959528953302853, 0.06666659269188946]
    u = [124.17276299136248, 135.60429047063715, 145.9949664404939, 24.18838184023805]
    
    @test lmult(q2)*q1 ≈ load(resfile, "q2q1") rtol=1e-6          # POINTS = 0.5
    @test lmult(q1)'q1 ≈ [1,0,0,0] rtol=1e-8                      # POINTS = 0.5
    
    dx = state_error(model, x1, x2)
    @test dx ≈ load(resfile, "dx") rtol=1e-6                      # POINTS = 2

    G0 = attitude_jacobian(q1)
    @test G0 ≈ load(resfile, "G1")                                # POINTS = 1
    
    G = state_error_jacobian(model, x1)
    @test G*dx ≈ load(resfile, "Gdx") rtol=1e-6                   # POINTS = 1
end;

Test Summary: | Pass  Total
Part b        |    5      5


## Part (c): A Better Cost Function (6 pts)
In our trajectory optimization problem, we're going to want to penalize distance from a reference trajectory or a goal, so we need a cost function that does this efficiently and correctly for our state vector, which includes quaternions. While we could use the error we computed above, this is very nonlinear and can impede convergence. Instead, we use the simple *geodesic distance* between two quaternions:
$$ \text{dist}_\text{geo}(q_1,q_2) = \min(1 \pm q_1^T q_2) $$

For our trajectory optimization problem we're going to use a cost function of the following form:
$$ \frac{1}{2}\left( (p-p_d)^T Q_p (p-p_d) + (v-v_d)^T Q_v (v-v_d) + (\omega - \omega_d)^T Q_\omega (\omega - \omega_d) \right)  + w \cdot \text{dist}_\text{geo}(q, q_d)$$
Use this to flush out the cost function below, and it's derivatives.

**NOTE**: You will be deducted points if you use FiniteDiff or ForwardDiff (or related packages) to compute the derivatives.

In [ ]:
# TASK: Complete the following methods
#       1. stagecost (1 pt)
#       2. termcost  (2 pts)
#       3. gradient  (2 pts)
#       4. hessian   (1 pt)

"""
    RigidBodyCost{m,T}

A cost function that penalizes distance of a rigid body state specified by its 
position `p`, attitude `q` (a unit quaternion), linear velocity `v` and angular velocity `ω`.

The vector state use a standard quadratic penalty, and the attitude uses the geodesic distance, weighted by a scalar `w`.
Uses a quadratic penalty on the distance of the `m`-dimensional control vector from a reference control `u_ref`.
"""
struct RigidBodyCost{m,T}
    Qp::Diagonal{T,SVector{3,T}}
    w::T
    Qv::Diagonal{T,SVector{3,T}}
    Qω::Diagonal{T,SVector{3,T}}
    R::Diagonal{T,SVector{m,T}}
    p_ref::SVector{3,T}
    q_ref::SVector{4,T}
    v_ref::SVector{3,T}
    ω_ref::SVector{3,T}
    u_ref::SVector{m,T}
end

"""
    stagecost(cost::RigidBodyCost, x, u)

Calculate the cost for a single knot point for the `RigidBodyCost`. 
TIP: Call `termcost` and add the control cost, to avoid repeated code
"""
function stagecost(cost::RigidBodyCost, x, u)
    # TODO: Calculuate the stage cost
    J = NaN
    
    return J
end

"""
    termcost(cost::RigidBodyCost, x)

Calculate the state-only terminal cost for a `RigidBodyCost`. 
Should use the geodesic distance to compute the error between the 
quaternion in `x` and `cost.q_ref`.
"""
function termcost(cost::RigidBodyCost, x)
    # TODO: Calculate the state-only cost
    J = NaN
    return J
end

"""
    gradient(cost::RigidBodyCost, x, u)

Calculate the gradient of the cost function with respect to
the states `x` and controls `u`. Should return both as a tuple.
"""
function gradient(cost::RigidBodyCost, x, u)
    # TODO: Calculate the gradients with respect to both x and u
    grad_x = zero(x)*NaN
    grad_u = zero(u)*NaN
    
    return grad_x, grad_u
end

"""
    hessian(cost::RigidBodyCost, x, u)

Calculate the Hessian of the cost function with respect to both the 
states `x` and controls `u` for a `RigidBodyCost`. 
Should return a tuple of both Hessians
"""
function hessian(cost::RigidBodyCost, x, u)
    # TODO: Calculate Cost Hessian
    Q = @SMatrix ones(length(x), length(x))
    R = @SMatrix ones(length(u), length(u))
    
    return Q, R
end

"""
    cost(obj, X, U)

Calculate the cost along an entire trajectory using a collection
of cost function types (stored in the iteratable collection `obj`).
"""
function cost(obj, X, U)
    J = zero(eltype(X[1]))
    for k = 1:length(U)
        J += stagecost(obj[k], X[k], U[k])
    end
    J += termcost(obj[end], X[end])
    return J
end

In [ ]:
# Test cost and derivatives
@testset "Part c" begin                                                                      # POINTS = 6
    model = YakPlane(UnitQuaternion)
    x = SA[0.45493888030931307, 0.8214867872560794, 0.587845774939719, 0.0859756782429905, -0.357328766626224, -0.4039767922209037, -0.8376915223630642, 0.274895624769145, 0.40424715886322726, 0.7686052831243393, 0.1958635140022562, 0.33959528953302853, 0.06666659269188946]
    u = SA[124.17276299136248, 135.60429047063715, 145.9949664404939, 24.18838184023805]
    ip,iq,iv,iw = state_parts(model)
    costfun = RigidBodyCost(
        Diagonal(@SVector fill(0.1, 3)),
        10.0, 
        Diagonal(@SVector fill(0.1, 3)),
        Diagonal(@SVector fill(0.1, 3)),
        Diagonal(@SVector fill(1e-3, 4)),
        SA[0,0,0.],
        SA[1,0,0,0.],
        SA[5,0,0.],
        SA[0,0,0.],
        u
    )
    
    J1 = stagecost(costfun, x, u)
    xflip = [x[ip]; -x[iq]; x[iv]; x[iw]]
    @test stagecost(costfun, xflip, u) ≈ J1                                                  # POINTS = 1
    @test J1 ≈ load(resfile, "J1") rtol=1e-5                                                 # POINTS = 1
    
    Jterm = termcost(costfun, x)
    @test Jterm ≈ load(resfile, "Jterm") atol=1e-8 rtol=1e-5                                 # POINTS = 1

    @test ForwardDiff.gradient(x->termcost(costfun,x), x) ≈ gradient(costfun, x, u)[1]       # POINTS = 1
    @test ForwardDiff.gradient(u->stagecost(costfun,x,u), u) ≈ gradient(costfun, x, u)[2]    # POINTS = 1
    @test ForwardDiff.hessian(x->termcost(costfun,x), x) ≈ hessian(costfun, x, u)[1]         # POINTS = 0.5
    @test ForwardDiff.hessian(u->stagecost(costfun,x,u), u) ≈ hessian(costfun, x, u)[2]      # POINTS = 0.5
end;

## Part (d): Adapt the iLQR Algorithm (12 pts)
Now that we have all the building blocks, modify the iLQR algorithm to work with the airplane dynamics. You'll need to make modifications in two places:
1. The backward pass: You'll need to "convert" the Jacobians to be on the error state, by multiplying by the error state Jacobian. You'll need to handle the gradients, Hessians (remember the extra term) and dynamics Jacobians all slightly differently.
2. The forward pass: when simulating the dynamics forward, you'll need to apply the control law on the error state.

In [ ]:
"""
    iLQRProblem{n,m,L}

Describes a trajectory optimization problem with `n` states, `m` controls, and 
a model of type `L`. 

# Constructor
    Problem(model::L, obj::Vector{<:QuadraticCost{n,m}}, tf, x0, xf) where {n,m,L}

where `tf` is the final time, and `x0` is the initial state. 
"""
struct iLQRProblem{n,m,L,O}
    model::L
    obj::Vector{O}
    N::Int
    tf::Float64
    x0::MVector{n,Float64}
    times::Vector{Float64}
    function iLQRProblem(model::L, obj::Vector{O}, tf, x0) where {L,O}
        n,m = size(model)
        @assert length(x0) == n
        T = length(obj)
        times = range(0, tf, length=T)
        new{n,m,L,O}(model, obj, T, tf, x0, times)
    end
end
Base.size(prob::iLQRProblem{n,m}) where {n,m} = (n,m,prob.N)

In [ ]:
# TASK: Modify the backward pass to calculate the control law on the error state (8 pts)
"""
    backwardpass!(prob, P, p, K, d, X, U)

Evaluate the iLQR backward pass at state and control trajectories `X` and `U`, 
storing the cost-to-go expansion in `P` and `p` and the gains in `K` and `d`.

Should return ΔJ, expected cost reduction.
"""
function backwardpass!(prob::iLQRProblem{n,m}, P, p, K, d, X, U; 
        β=1e-6
    ) where {n,m}
    N = prob.N
    ΔJ = 0.0
    model = prob.model
    
    iq = state_parts(prob.model)[2]
    Iq = Diagonal(SA[0,0,0,1,1,1, 0,0,0, 0,0,0])

    # Calculate gradient and Hessian at terminal state
    Q, = hessian(prob.obj[N], X[N], 0*U[1])
    q, = gradient(prob.obj[N], X[N], 0*U[1])
    
    # TODO: Calculate the terminal cost-to-go, accounting for the group structure of the 3D rotation
    p[N]
    P[N]
    
    
    #Backward Pass
    for k = (N-1):-1:1

        # Cost Expansion
        Q,R =  hessian(prob.obj[k], X[k], U[k])
        q,r = gradient(prob.obj[k], X[k], U[k])

        # Dynamics derivatives
        dt = prob.times[k+1] - prob.times[k]
        A,B = discrete_jacobian(model, X[k], U[k], prob.times[k], dt)

        # TODO: Account for the group structure of 3D rotations using the attitude Jacobian (via state_error_jacobian)
        #  TIP: A,B,Q, and q should all be modified (including their size)
        
    
        gx = q + A'*p[k+1]
        gu = r + B'*p[k+1]
    
        Gxx = Q + A'*P[k+1]*A
        Guu = R + B'*P[k+1]*B
        Gux = B'*P[k+1]*A
        
        # Regularization
        Gux_reg = SMatrix{m,n-1}(Gux)
        Guu_reg = SMatrix{m,m}(Guu + β*Diagonal(@SVector ones(m)))
        
        # Calculate Gains
        F = cholesky(Symmetric(Guu_reg))
        d[k] .= (F\gu)
        K[k] .= (F\Gux_reg)
    
        # Cost-to-go Recurrence
        p[k] .= gx - K[k]'*gu + K[k]'*Guu*d[k] - Gux'*d[k]
        P[k] .= Gxx + K[k]'*Guu*K[k] - Gux'*K[k] - K[k]'*Gux
        ΔJ += gu'*d[k]
   
    end
    return ΔJ
end

In [ ]:
# TASK: Modify the forward pass algorithm to apply the control policy on the error state (4 pts)
"""
    forwardpass!(prob, X, U, K, d, ΔJ, J)

Evaluate the iLQR forward pass at state and control trajectories `X` and `U`, using
the gains `K` and `d` to simulate the system forward. The new cost should be less than 
the current cost `J` together with the expected cost decrease `ΔJ`.

Should return the new cost `Jn` and the step length `α`.
"""
function forwardpass!(prob::iLQRProblem{n,m}, X, U, K, d, ΔJ, J,
        Xbar = deepcopy(X), Ubar = deepcopy(U);
        max_iters=10,
    ) where {n,m}
    N = prob.N
    model = prob.model

    Jn = J
    α = 0.0
    
    # Line Search
    Xbar[1] = X[1]
    α = 1.0
    Jn = Inf
    for i = 1:max_iters
        
        # Forward Rollout
        for k = 1:(N-1)
            t = prob.times[k]
            dt = prob.times[k+1] - prob.times[k]
            
            # TODO: apply the local feedback control on the error state
        end
        
        # Calculate the new cost
        Jn = cost(prob.obj, Xbar, Ubar)

        # Check Armijo condition
        if Jn <= J - 1e-2*α*ΔJ
            break
        else
            # Backtrack
            α *= 0.5  
        end
        if i == max_iters 
            α = 0
        end
    end
    
    # Accept direction
    for k = 1:N-1
        X[k] = Xbar[k]
        U[k] = Ubar[k]
    end
    X[N] = Xbar[N]
    
    return Jn, α
end

In [ ]:
"""
    solve_ilqr(prob, X, U; kwargs...)

Solve the trajectory optimization problem specified by `prob` using iterative LQR.
Returns the optimized state and control trajectories, as well as the local control gains,
`K` and `d`.

Should return the optimized state and control trajectories `X` and `U`, and the 
list of feedback gains `K` and cost-to-go hessians `P`.
"""
function solve_ilqr(prob::iLQRProblem{n,m}, X0, U0; 
        iters=100,     # max iterations
        ls_iters=10,   # max line search iterations
        reg_min=1e-6,  # minimum regularizatio for the backwardpass
        verbose=0,     # print verbosity
        eps=1e-5,      # termination tolerance
        eps_ddp=eps    # tolerance to switch to ddp
    ) where {n,m}
    t_start = time_ns()
    Nx,Nu,Nt = size(prob)

    # Initialization
    # NOTE: some of these sizes have changed!
    N = prob.N
    p = [zeros(n-1) for k = 1:N]              # ctg gradient
    P = [zeros(n-1,n-1) for k = 1:N]          # ctg hessian
    d = [zeros(m) for k = 1:N-1]              # feedforward gains
    K = [zeros(m,n-1) for k = 1:N-1]          # feedback gains
    Xbar = [@SVector zeros(n) for k = 1:N]    # line search trajectory
    Ubar = [@SVector zeros(m) for k = 1:N-1]  # line search trajectory
    ΔJ = 0.0

    # Don't modify the trajectories that are passed in
    X = deepcopy(X0)
    U = deepcopy(U0)

    # Initial cost
    J = cost(prob.obj, X, U)
    
    # Initialize parameters
    Jn = Inf
    iter = 0
    tol = 1.0
    β = reg_min
    while tol > eps 
        iter += 1
        
        # Backward Pass
        ΔJ, = backwardpass!(prob, P, p, K, d, X, U, β=β)

        # Forward Pass
        Jn, α = forwardpass!(prob, X, U, K, d, ΔJ, J, Xbar, Ubar, max_iters=ls_iters)

        if α === zero(α) 
            β = max(β*10, 1.0)
            # β *= 10 
        else 
            β = max(β/2, reg_min)
        end

        # Update parameters
        tol = maximum(norm.(d, Inf))
        β = max(0.9*β, reg_min)

        # Output
        if verbose > 0
            @printf("Iter: %3d, Cost: % 6.2f → % 6.2f (% 7.2e), res: % .2e, β= %.2e, α = %.3f, ΔJ = %.3e\n",
                iter, J, Jn, J-Jn, tol, β, α, ΔJ
            )
        end
        J = Jn

        if iter >= iters
            @warn "Reached max iterations"
            break
        end

    end
    println("Total Time: ", (time_ns() - t_start)*1e-6, " ms")
    return X,U,K,P
end

In [ ]:
using RobotDynamics: RBState
function YakProblems(;
        N = 101,
        vecstate=false,
        scenario=:barrellroll, 
        heading=0.0,  # deg
        Qpos=1.0,
        kwargs...
    )
    model = YakPlane(UnitQuaternion)

    n,m = size(model)
    ip,iq,iv,iw = state_parts(model)

    # Discretization
    tf = 1.25
    if scenario == :fullloop 
        # Double the time, keeping the same discretization
        tf *= 2
        N = (N-1)*2 + 1
    end
    dt = tf/(N-1)

    # Forward Velocity 
    vel = 5.0

    if scenario ∈ (:halfloop, :fullloop) 
        ey = @SVector [0,1,0.]

        # Heading
        dq = expm(SA[0,0,1]*deg2rad(heading))

        # Initial state
        p0 = MRP(0.997156, 0., 0.075366) # initial orientation (level flight)
        x0 = RD.build_state(model, [-3,0,1.5], p0, [vel,0,0], [0,0,0])

        # Climb
        pm = expm(SA[1,0,0]*deg2rad(180))*expm(SA[0,1,0]*deg2rad(90))
        xm = RD.build_state(model, [0,0,3.], pm, pm * [vel,0,0.], [0,0,0])

        # Top of loop
        pf = MRP(0., -0.0366076, 0.) * dq # final orientation (upside down)
        xf = RD.build_state(model, dq*[3,0,6.], pf, pf * [vel,0,0.], [0,0,0])
        pf2 = RotZ(deg2rad(heading-180))

        # Dive
        xm2 = RD.build_state(model, [-3,3,4.], pm * RotY(pi), [0,0,-vel], [0,0,0])

        # Terminal state
        xf2 = RD.build_state(model, [0,4,1.5], p0, [vel,0,0], [0,0,0])

        t_flat = 5 / (xf[2] - xf[1])
        N_flat = Int(round(t_flat/dt))

        # Xref trajectory
        x̄0 = RBState(model, x0)
        x̄m = RBState(model, xm)
        x̄m2 = RBState(model, xm2)
        x̄f = RBState(model, xf)
        x̄f2 = RBState(model, xf2)
        Xref = map(1:N) do k
            t = (k-1)/(N-1)
            Nmid = N ÷ 4
            if scenario == :fullloop
                if k < Nmid
                    x1 = x̄0
                    x2 = x̄m
                    t = (k-1)/Nmid
                elseif k < 2Nmid
                    t = (k-Nmid)/Nmid
                    x1 = x̄m
                    x2 = x̄f 
                elseif k < 3Nmid
                    t = (k-2Nmid)/Nmid
                    x1 = x̄f
                    x2 = x̄m2
                else
                    t = (k-3Nmid)/Nmid
                    x1 = x̄m2
                    x2 = x̄f2
                end
            else
                if k < 2Nmid
                    t = (k-1)/2Nmid
                    x1 = x̄0
                    x2 = x̄m
                else
                    t = (k-2Nmid)/(2Nmid+1)
                    x1 = x̄m
                    x2 = x̄f
                end
            end
            RBState(
                x1.r + (x2.r - x1.r)*t,
                slerp(x1.q, x2.q, t),
                x1.v + (x2.v - x1.v)*t,
                SA[0,pi/1.25,0]
            )
        end
    else
        throw(ArgumentError("$scenario isn't a known scenario"))
    end

    # Get trim condition
    utrim = get_trim(model, x0, fill(124, 4))

    # Objective
    Qf_diag = RD.fill_state(model, 10, 500*0, 100, 100.)
    Q_diag = RD.fill_state(model, Qpos*0.1, 0.1*0, 0.1, 1.1)
    R = Diagonal(@SVector fill(1e-3,4))
    costs = map(1:N-1) do k
        RigidBodyCost(
            Diagonal(Q_diag[ip])*dt,
            10.0*dt, 
            Diagonal(Q_diag[iv])*dt,
            Diagonal(Q_diag[iw])*dt,
            R*dt,
            Xref[k][ip],
            Xref[k][iq],
            Xref[k][iv],
            Xref[k][iw],
            SVector{4,Float64}(utrim)
        )
    end
    costterm = RigidBodyCost(
            Diagonal(Qf_diag[ip]),
            200.0, 
            Diagonal(Qf_diag[iv]),
            Diagonal(Qf_diag[iw]),
            R*0,
            Xref[N][ip],
            Xref[N][iq],
            Xref[N][iv],
            Xref[N][iw],
            SVector{4,Float64}(utrim)
        )
    push!(costs, costterm)

    # Build problem
    prob = iLQRProblem(model, costs, tf, x0)

    return prob, Xref, utrim
end

## Part (e): Solve for half of the loop (2 pts)
It turns out the maneuver we're trying to execute is a little tricky to get with a simple trim condition initializer (remember that these problems have lots of local minima). To get the desired behavior, we're going to first solve a simpler problem that will give us a good initialization for the problem we actually care about. We're basically going to solve for half of the loop first, and then ask it to get the rest of the loop.

We've taken care of the reference trajectories for this problem, but we encourage you to look at the code and at the trajectories in the visualizer.

#### Visualizer

In [ ]:
if !isautograder
    vis = Visualizer()
    TrajOptPlots.set_mesh!(vis, YakPlane(UnitQuaternion))
    render(vis)  # this may take a while, it's a detailed model
end

In [ ]:
# Generate the problem, and visualize the reference trajectory
prob_half, Xref_half = YakProblems(costfun=:QuatLQR, scenario=:halfloop, heading=130)
isautograder || visualize!(vis, prob_half.model, prob_half.tf, Xref_half)

In [ ]:
# TASK: Generate the initial guess, using trim conditions (2 pts)
#  TIP: calculate the trim controls using get_trim
#  TIP: write a method to simulate the dynamics forward

U0 = [@SVector zeros(control_dim(prob_half.model)) for k = 1:prob_half.N-1]  # TODO: Update this!
X0 = [SVector(prob_half.x0) for k = 1:prob_half.N]      # remember this needs to be dynamically feasible!


isautograder || visualize!(vis, prob_half.model, prob_half.tf, X0)

In [ ]:
# Solve for the first half of the trajectory
# TIP: It should take about 17 iterations
# TIP: Final cost should be about 6.2

Xhalf, Uhalf = solve_ilqr(prob_half, X0, U0, verbose=1, eps=1e-4, reg_min=1e-6)
isautograder || visualize!(vis, prob_half.model, prob_half.tf, Xhalf)

In [ ]:
@testset "Part d" begin                                                                 # POINTS = 12
    let prob = prob_half
        n,m,N = size(prob)
        p = [zeros(n-1) for k = 1:N]              # ctg gradient
        P = [zeros(n-1,n-1) for k = 1:N]          # ctg hessian
        d = [zeros(m) for k = 1:N-1]              # feedforward gains
        K = [zeros(m,n-1) for k = 1:N-1]          # feedback gains
        Xbar = [@SVector zeros(n) for k = 1:N]    # line search trajectory
        Ubar = [@SVector zeros(m) for k = 1:N-1]  # line search trajectory

        X = deepcopy(X0)
        U = deepcopy(U0)

        # Initial cost
        J = cost(prob.obj, X, U)
        @test J ≈ load(resfile, "J0")                                                   # POINTS = 1
        
        # Backward Pass
        ΔJ, = backwardpass!(prob, P, p, K, d, X, U, β=1e-8)
        @test ΔJ ≈ load(resfile, "dJ") rtol=1e-3                                        # POINTS = 5
        @test norm(d,Inf) ≈ load(resfile, "d0") rtol=1e-3                               # POINTS = 2

        # Forward Pass
        Jn, α = forwardpass!(prob, X, U, K, d, ΔJ, J, Xbar, Ubar, max_iters=10)
        @test Jn < J                                                                    # POINTS = 2
        @test (J-Jn) / ΔJ ≈ 0.025 atol=1e-3                                             # POINTS = 1
        @test Jn ≈ load(resfile, "Jn") rtol=1e-3                                        # POINTS = 1
    end
    
end;

In [ ]:
@testset "Part e" begin                                                                 # POINTS = 2
    model = prob_half.model
    @test norm(RBState(model,Xhalf[end]) ⊖ RBState(model,Xref_half[end])) < 0.5
    @test cost(prob_half.obj, Xhalf, Uhalf) < 6.5
    let prob = prob_half, X=Xhalf, U=Uhalf
        n,m,N = size(prob)
        p = [zeros(n-1) for k = 1:N]              # ctg gradient
        P = [zeros(n-1,n-1) for k = 1:N]          # ctg hessian
        d = [zeros(m) for k = 1:N-1]              # feedforward gains
        K = [zeros(m,n-1) for k = 1:N-1]          # feedback gains
        ΔJ, = backwardpass!(prob, P, p, K, d, X, U, β=1e-6)
        @test ΔJ < 1e-4                                                                  # POINTS = 1
        @test norm(d) < 0.1                                                              # POINTS = 1
    end
end;

## Part (f): Complete the Loop (3 pts)
Use the result of your previous solve to generate a good initial guess for the full maneuver. The new trajectory is twice as long as the previous one.

**TIP**: Reverse the control sequence for the second half of the trajectory.

In [ ]:
# Generate the new problem and reference trajectory
prob, Xref_full = YakProblems(costfun=:QuatLQR, scenario=:fullloop, heading=130, Qpos=100)
isautograder || render(vis)

In [ ]:
# Visualize the reference trajectory
isautograder || visualize!(vis, prob.model, prob.tf, Xref_full)

In [ ]:
# TASK: Design a good initialization, using result for the previous problem (2 pts)
U0 = [copy(utrim) for k = 1:prob_half.N-1]  # TODO: update this
X0 = rollout(prob.model, prob.x0, U0, prob.times)


isautograder || visualize!(vis, prob.model, prob.tf, X0)

In [ ]:
# Solve for the full loop
# TIP: This will probably take between 100-200 iterations
# TIP: Final cost should be about 14
# TIP: the regularization parameter has already been tuned, but feel free to change it if needed
# NOTE: We've relaxed the optimality condition for this solve, to make it easier

Xfull, Ufull = solve_ilqr(prob, X0, U0, verbose=1, eps=1e-2, reg_min=1e-8, iters=500)
isautograder || visualize!(vis, prob.model, prob.tf, Xfull)

In [ ]:
@testset "Part f" begin                                                                     # POINTS = 3
    @test norm(RBState(prob.model,Xfull[end]) ⊖ RBState(prob.model,Xref_full[end])) < 0.5
    @test cost(prob.obj, Xfull, Ufull) < 14.5
    let prob = prob, X=Xfull, U=Ufull
        n,m,N = size(prob)
        p = [zeros(n-1) for k = 1:N]              # ctg gradient
        P = [zeros(n-1,n-1) for k = 1:N]          # ctg hessian
        d = [zeros(m) for k = 1:N-1]              # feedforward gains
        K = [zeros(m,n-1) for k = 1:N-1]          # feedback gains
        ΔJ, = backwardpass!(prob, P, p, K, d, X, U, β=1.0)
        @test ΔJ < 1.0                                                                      # POINTS = 1
        @test norm(d) < 1.0                                                                 # POINTS = 2
    end
end;